In [ ]:
import optuna 
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracyscore


# optuna가 한번 trial(시도) 할떄마다 하이퍼 파라미터를 제안하는 범위를 지정한다
# trial.suggest : optuna 가 값을 뽑는 함수 (GridSearchCV에서 "그리드" 역할)
def objective(trial):
    params = {
        "max_depth" : trial.suggest_int("max_depth", 2, 10),                    # 트리 깊이 : 2 ~. 10 중 선택
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),     # 분기 최소 샘플 수. : 2 ~ 10 중 선택
        "min_samples_leaf" : trial.suggest_int("min_samples_leaf", 1, 10),      # 리프 최소 샘플 수.  1~ ~ 10 중 선택
        "random_state" : 11 # 재현성 (시드 고정)
    }

    model = DecisionTreeClassifier(params) # 모델 생성( (unpacking) : 딕셔너리 key= value 쌍을 펼쳐서 넣는다)

    # 교차검증 설정 (StratifiedKFold: fold 마다 클래스 비율을 유지. 불균형 데이터에 강하다 🙂
    cv = StratifiedKFold(
        n_splits=5,         # 5등분
        shuffle= True,      # 데이터 랜덤으로 사용
        random_state= 11    # 섞는 방식은 고정 ( 재현성 ) 
    )

    # 학습 폴드로는 fit , 검증 폴드로는 predict, scoring 기준으로 점수 계산 5번 반복 후 점수 배열 반환
    scores = cross_val_score(model, X_train, y_train, cv = cv, scoring = 'accuracy')


    return scores.mean() # 평균 정확도를 반환. => optuna가 최대화할 값 반환 (평균 정확도가 높아지는 방향으로 파라미터를 잡는다 )

# Optuna 스터디 생성 ( direction = "maximize" : objective 가 반환하는 값(정확도)를 최대화 하도록 탐색)
study = optuna.create_study(direction = "maximize")

#최적화 수행
study.optimize(
    objective,          # 함수(파라미터를 업데이트 하는데 참고하는 함수)
    n_trials = 50       # 파라미터 조합 시도 수 
    )

# 탐색 결과 출력
print("Optuna 최적 하이퍼파라미터:", study.best_params)
print(f"Optuna 교차 검증 최고 정확도: {study.best_value:.4f}")

# 최적 파라미터 적용 (best_params 는 딕셔너리 형태이기 떄문에 unpacking)
best_optuna_df = DecisionTreeClassifier(**study.best_params, random_state= 11) 
best_optuna_df.fit(X_train,y_train)

# 평가
optuna_pred = best_optuna_df.predict(X_test)
optuna_acc =accuracy_score(y_test, optuna_pred )
print(f"테스트 셋으로 평가한 Optuna 튜닝DecisionTreeClassifier 정확도:{optuna_acc:.4f}")